In [1]:
!pip install -U imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 199 kB 7.7 MB/s 
     |████████████████████████████████| 199 kB 34.0 MB/s 
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.8.1
    Uninstalling imbalanced-learn-0.8.1:
      Successfully uninstalled imbalanced-learn-0.8.1


In [13]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [74]:
df = pd.read_csv('/content/drive/MyDrive/overpass_knn3_drop.csv', encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                183 non-null    int64  
 1   위도                183 non-null    float64
 2   경도                183 non-null    float64
 3   생태통로_유형           183 non-null    int64  
 4   등산객_이용빈도          183 non-null    float64
 5   생태통로_효율성          183 non-null    int64  
 6   이용확인종_수           183 non-null    int64  
 7   폭(m)              183 non-null    float64
 8   연장(m)             183 non-null    float64
 9   경사도               183 non-null    int64  
 10  주변 로드킬 빈도         183 non-null    int64  
 11  산책로까지의 최단 거리(km)  183 non-null    float64
 12  농가까지의 거리(km)      183 non-null    float64
 13  도로 최고제한속도(km/h)   183 non-null    int64  
 14  주변 동물종 개수         183 non-null    float64
 15  주변동물 출현빈도         183 non-null    float64
 16  지형기호(2.3km)       183 non-null    object 
 1

In [75]:
df.drop(['최우점식물군락'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                183 non-null    int64  
 1   위도                183 non-null    float64
 2   경도                183 non-null    float64
 3   생태통로_유형           183 non-null    int64  
 4   등산객_이용빈도          183 non-null    float64
 5   생태통로_효율성          183 non-null    int64  
 6   이용확인종_수           183 non-null    int64  
 7   폭(m)              183 non-null    float64
 8   연장(m)             183 non-null    float64
 9   경사도               183 non-null    int64  
 10  주변 로드킬 빈도         183 non-null    int64  
 11  산책로까지의 최단 거리(km)  183 non-null    float64
 12  농가까지의 거리(km)      183 non-null    float64
 13  도로 최고제한속도(km/h)   183 non-null    int64  
 14  주변 동물종 개수         183 non-null    float64
 15  주변동물 출현빈도         183 non-null    float64
 16  지형기호(2.3km)       183 non-null    object 
 1

In [76]:
df['식물군락명'] = [0 if ((not '나무' in x) | (not '군락' in x)) else x for x in df['식물군락명']]
df['식물군락명'] = [1 if x!=0 else x for x in df['식물군락명']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                183 non-null    int64  
 1   위도                183 non-null    float64
 2   경도                183 non-null    float64
 3   생태통로_유형           183 non-null    int64  
 4   등산객_이용빈도          183 non-null    float64
 5   생태통로_효율성          183 non-null    int64  
 6   이용확인종_수           183 non-null    int64  
 7   폭(m)              183 non-null    float64
 8   연장(m)             183 non-null    float64
 9   경사도               183 non-null    int64  
 10  주변 로드킬 빈도         183 non-null    int64  
 11  산책로까지의 최단 거리(km)  183 non-null    float64
 12  농가까지의 거리(km)      183 non-null    float64
 13  도로 최고제한속도(km/h)   183 non-null    int64  
 14  주변 동물종 개수         183 non-null    float64
 15  주변동물 출현빈도         183 non-null    float64
 16  지형기호(2.3km)       183 non-null    object 
 1

In [77]:
df['지형기호(2.3km)'].value_counts()

X          94
하천-퇴적      39
산지-침식풍화    16
하천-침식       7
하천          7
산지-퇴적       7
산지          4
해안-퇴적       4
기타          3
해안-침식풍화     2
Name: 지형기호(2.3km), dtype: int64

In [78]:
df['지형기호(2.3km)'] = ['0' if (('X'==x) | ('기타'==x)) else x for x in df['지형기호(2.3km)']]
df['지형기호(2.3km)'] = ['1' if '하천' in x else x for x in df['지형기호(2.3km)']]
df['지형기호(2.3km)'] = ['2' if '산지' in x else x for x in df['지형기호(2.3km)']]
df['지형기호(2.3km)'] = ['3' if '해안' in x else x for x in df['지형기호(2.3km)']]

In [79]:
df['지형기호(2.3km)']=df['지형기호(2.3km)'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                183 non-null    int64  
 1   위도                183 non-null    float64
 2   경도                183 non-null    float64
 3   생태통로_유형           183 non-null    int64  
 4   등산객_이용빈도          183 non-null    float64
 5   생태통로_효율성          183 non-null    int64  
 6   이용확인종_수           183 non-null    int64  
 7   폭(m)              183 non-null    float64
 8   연장(m)             183 non-null    float64
 9   경사도               183 non-null    int64  
 10  주변 로드킬 빈도         183 non-null    int64  
 11  산책로까지의 최단 거리(km)  183 non-null    float64
 12  농가까지의 거리(km)      183 non-null    float64
 13  도로 최고제한속도(km/h)   183 non-null    int64  
 14  주변 동물종 개수         183 non-null    float64
 15  주변동물 출현빈도         183 non-null    float64
 16  지형기호(2.3km)       183 non-null    int64  
 1

#SMOTE



In [80]:
# x, y
x=df.drop('생태통로_효율성', axis=1)
y=df['생태통로_효율성']

# train & test set
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, stratify=df['생태통로_효율성'])

print('SMOTE 적용 전 값의 분포 :\n',pd.Series(train_y).value_counts())

SMOTE 적용 전 값의 분포 :
 1    73
0    73
Name: 생태통로_효율성, dtype: int64


In [81]:
train_over_x, train_over_y = SMOTE(random_state=42).fit_resample(train_x, train_y)
print("SMOTE 적용 전 학습용 피처/레이블 데이터 세트 : ", train_x.shape, train_y.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트 :', train_over_x.shape, train_over_y.shape)
print('SMOTE 적용 후 값의 분포 :\n',pd.Series(train_over_y).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트 :  (146, 27) (146,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트 : (146, 27) (146,)
SMOTE 적용 후 값의 분포 :
 1    73
0    73
Name: 생태통로_효율성, dtype: int64


결과 : train_y의 0,1 클래스의 비율이 동일해 데이터 증식이 되지 않습니다.